In [35]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service


from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from webdriver_manager.microsoft import EdgeChromiumDriverManager



from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select




from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException, StaleElementReferenceException, NoSuchElementException
import time
import traceback

In [36]:
import pandas as pd
from datetime import datetime


pos = 1
autentica = pd.read_excel('ls.xlsx')
login = str(autentica['login'][pos])

autentica['senha'] = autentica['senha'].astype(str)
autentica['senha'] = autentica['senha'].str.zfill(11)

senha = autentica['senha'][pos]

display(autentica)
name = str(autentica['ubs'][pos])
data_e_hora_atuais = datetime.now()
data_e_hora = data_e_hora_atuais.strftime('%Y-%m-%d_%H-%M')

raceList = ['BRANCA', 'PRETA', 'AMARELA', 'PARDA', 'INDÍGENA', 'SEM INFORMAÇÃO']


# nav = webdriver.Chrome(ChromeDriverManager().install())
# nav = webdriver.Edge(EdgeChromiumDriverManager().install())


,login,senha,ubs
0,joana,05216791418,borges
1,agda,09947741419,dois_riachos


In [37]:
def openEpHealph():
    
#     nav = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
#     nav = webdriver.Chrome(ChromeDriverManager().install())
    global nav
    nav = webdriver.Firefox(executable_path=GeckoDriverManager().install())
    nav.get("https://afogadosdaingazeira.ephealth.com.br/portal/login")
    
    # inserindo usuário
    nav.find_element(By.ID, 'EMail').click()
    nav.find_element(By.ID, 'EMail').send_keys(login)

    # inserindo senha 
    nav.find_element(By.ID, 'Password').click()
    nav.find_element(By.ID, 'Password').send_keys(senha)

    nav.find_element(By.XPATH, '/html/body/form/button').click()

In [38]:
def people(has_dm):
    
    openEpHealph()
        
    print(f'Extração iniciada as: {datetime.now()}')

    nav.get("https://afogadosdaingazeira.ephealth.com.br/portal/individuo")
    
    base = nav.find_element(By.XPATH, has_dm).click()

#     children = nav.find_element(By.ID, 'maxAge')
#     nav.execute_script("arguments[0].setAttribute('value', '4.0')",children)
#     time.sleep(1)
    
#     hypertension = nav.find_element(By.XPATH, '//*[@id="img_hipertensaoFlag"]').click()
#     time.sleep(1)

#     diabetes = nav.find_element(By.XPATH, '//*[@id="img_diabetesFlag"]').click()
#     time.sleep(1)

#     pregnant = nav.find_element(By.XPATH, '//*[@id="img_gestanteFlag"]').click()
#     time.sleep(1)

    """
    race = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/form/div[3]/div[5]/span/span[1]/span/ul/li/input')
    race.click()
    time.sleep(1)
    race.send_keys(raceList[race_item])
    race.send_keys(Keys.ENTER)
    """

    time.sleep(1)

    unity = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/form/div[4]/div[1]/span/span[1]/span/span[2]')
    unity.click()
    time.sleep(1)
    nav.find_element(By.XPATH,'/html/body/span/span/span[1]/input').send_keys(Keys.ENTER)
    time.sleep(1)
    nav.find_element(By.ID, 'button').click()

    time.sleep(1)
    nav.find_element(By.ID, 'card-toggle').click()
    time.sleep(10)

In [39]:
def get_table(has_dm):
    
    list_links = []
    people(has_dm)
    
    print(f'Iniciando lista de links: {datetime.now()}')

    exit = False
    
    peoples = nav.find_element(By.CLASS_NAME, 'dataTables_scrollBody')
    link = peoples.find_elements(By.TAG_NAME, 'a') 
    
    
    
    while exit is False:
        try:
            for a in link:
                
                list_links.append(a.get_attribute('href'))
                
            page = nav.find_element(By.CSS_SELECTOR, f"#individuo-list-datatable_next > a:nth-child(1)")

            page.click()
            time.sleep(10)                
            
        except (ElementClickInterceptedException, NoSuchElementException):
#             if ElementClickInterceptedException:    
#                 traceback.print_exc()
#             if NoSuchElementException: 
#                 traceback.print_exc()
            print('acabaram as paginas')
            break
        
        except StaleElementReferenceException:
            peoples = nav.find_element(By.CLASS_NAME, 'dataTables_scrollBody')
            link = peoples.find_elements(By.TAG_NAME, 'a') 

    
    list_links = list(dict.fromkeys(list_links))
    print(f'lista de links finalizada as: {datetime.now()}')
    
    return list_links    


In [40]:
def get_individual_info(has_dm, p_has, p_dm):
    
#   name sex b_date cpf cns has dm med_rec race m_name, proffisional, unity
    name, sex, b_date, cpf, cns, has, dm, med_rec, race, m_name, profissional, unity = [], [], [], [], [], [], [], [], [], [], [], []

    
    individual = get_table(has_dm)
    
    print(f'Iniciando individuos : {datetime.now()}')
    for i in individual:
        
        time.sleep(1)
        
        nav.get(i)
        
        i_name = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/h5').text
        name.append(i_name)
        
        i_sex = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/p[1]').text
        i_sex = i_sex.split(',')
        sex.append(i_sex[0].upper())
        
        i_bdate = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[3]/div/div/table/tbody/tr[4]/td[2]').text
        i_bdate = i_bdate.split(";")
        b_date.append(i_bdate[0])
        
        linha_dois = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[2]/td[1]').text
        linha_tres = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[3]/td[1]').text
        
        linha_dois_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[2]/td[2]').text
        linha_tres_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[3]/td[2]').text
        
        if(linha_dois.upper() == "CPF"):
            cpf.append(linha_dois_text)
            
        elif(linha_tres.upper() == "CPF"):
            cpf.append(linha_tres_text)
            
        else:
            cpf.append("--")
        
        i_cns = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/p[2]').text
        i_cns = i_cns.split(':')
        if i_cns[0].upper() != 'CNS':
            cns.append('--')
        else:
            cns.append(i_cns[1])
        
        i_has = p_has
        has.append(i_has)
        
        i_dm = p_dm
        dm.append(i_dm)
        
        i_medrec = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[3]/div/div/table/tbody/tr[1]/td[2]').text
        med_rec.append(i_medrec)
        
        i_race = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[3]/div/div/table/tbody/tr[5]/td[2]').text
        race.append(i_race.upper())
        
        i_mname = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[3]/div/div/table/tbody/tr[6]/td[2]').text
        m_name.append(i_mname)
    
        try:
            i_uni = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/p[3]').text
        
        except NoSuchElementException:
            i_uni = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/p[2]').text
        
        i_uni = i_uni.split(':')
        unity.append(i_uni[1])
        
        try:
            linha_tres = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[3]/td[1]').text
            linha_tres_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[3]/td[2]').text
            if(linha_tres.upper() == 'PROFISSIONAL'):
                profissional.append(linha_tres_text)
        except:
            continue
        try:
            linha_quatro = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[4]/td[1]').text
            linha_quatro_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[4]/td[2]').text
            if(linha_quatro.upper() == 'PROFISSIONAL'):
                profissional.append(linha_quatro_text)
        except:
            continue
        
        try:
            linha_cinco = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[5]/td[1]').text
            linha_cinco_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[5]/td[2]').text
            if(linha_cinco.upper() == 'PROFISSIONAL'):
                profissional.append(linha_cinco_text)
        except:
            continue
        
        try:
            linha_seis = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[6]/td[1]').text
            linha_seis_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[6]/td[2]').text
            if(linha_seis.upper() == 'PROFISSIONAL'):
                profissional.append(linha_seis_text)
            
        except:
            continue
            
        time.sleep(1)
        
    print(f'Finalizando individuos : {datetime.now()}')

    dict_individual = {"name": name, "sex":sex, "b_date":b_date, "cpf":cpf, "cns":cns, "has":has, "dm":dm, "med_rec":med_rec, "race":race, "m_name":m_name, "profissional":profissional, "unity":unity, "link":individual}
    df_individual = pd.DataFrame(data=dict_individual)    
    
    print(f'Extração finalizada as: {datetime.now()}')   
    
    nav.quit()
    
    return df_individual

# indi = get_indivivual_info()

In [41]:
def get_has_dm():
    has_dm_click = ['//*[@id="img_hipertensaoFlag"]', '//*[@id="img_diabetesFlag"]']
    has = get_individual_info(has_dm_click[0], 'sim', 'nao')
    dm = get_individual_info(has_dm_click[1], 'nao', 'sim')
    
    has_dm = pd.concat([has, dm], ignore_index=True)
    
    double = has_dm[has_dm.duplicated(subset=['name', 'cpf'])]
    double.index = [x for x in range(0, len(double))]
    double['has'] = double['has'].replace(['nao'], 'sim')
    double['dm'] = double['dm'].replace(['nao'], 'sim')
    
    final = pd.concat([has_dm, double], ignore_index=True)
    final = final.drop_duplicates(subset=['name', 'cpf'], keep='last')
    final = final.sort_values(by=['name'])
    final.index = [x for x in range(0, len(final))]
    
    return final
    

In [42]:
unification = get_has_dm()

Extração iniciada as: 2024-01-26 16:12:56.250522
Iniciando lista de links: 2024-01-26 16:13:12.256222
acabaram as paginas
lista de links finalizada as: 2024-01-26 16:14:27.716046
Iniciando individuos : 2024-01-26 16:14:27.716077
Finalizando individuos : 2024-01-26 16:39:02.465814
Extração finalizada as: 2024-01-26 16:39:02.467101
Extração iniciada as: 2024-01-26 16:39:08.229384
Iniciando lista de links: 2024-01-26 16:39:24.053835
acabaram as paginas
lista de links finalizada as: 2024-01-26 16:39:35.111730
Iniciando individuos : 2024-01-26 16:39:35.111768
Finalizando individuos : 2024-01-26 16:44:59.231344
Extração finalizada as: 2024-01-26 16:44:59.232063


/tmp/ipykernel_9365/3913768521.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  double['has'] = double['has'].replace(['nao'], 'sim')
/tmp/ipykernel_9365/3913768521.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  double['dm'] = double['dm'].replace(['nao'], 'sim')


In [43]:
display(unification)
unification = unification[['name', 'sex', 'b_date', 'cpf', 'cns', 'has', 'dm', 'med_rec', 'race', 'm_name', 'link']]
unification.to_excel(f'{data_e_hora}_{name}_has_dm.xlsx', index=False)

,name,sex,b_date,cpf,cns,has,dm,med_rec,race,m_name,profissional,unity,link
0,ADAILTON RAMOS DE ARAUJO,MASCULINO,23/04/1976,--,701402612554733,sim,nao,295,PARDA,OLINDINA RAMOS DE ARAUJO,CECILIA MARIA DE OLIVEIRA,ESF DOIS RIACHOS,https://afogadosdaingazeira.ephealth.com.br/po...
1,ADELMO FLORIANO SIQUEIRA MOURA,MASCULINO,02/02/1962,37040499487,708104555877631,sim,nao,674,PARDA,MARIA DE LOURDES SIQUEIRA MOURA,MARIA DO ROSARIO SIQUEIRA,ESF DOIS RIACHOS,https://afogadosdaingazeira.ephealth.com.br/po...
2,ADELMO MORATO DE BRITO,MASCULINO,15/10/1957,30322383404,708202167366548,sim,nao,474,PARDA,QUITERIA HONORATO DE BRITO,DAMIANA DE OLIVEIRA BEZERRA PEREIRA,ESF DOIS RIACHOS,https://afogadosdaingazeira.ephealth.com.br/po...
3,AGNALDO COSME DE LIMA,MASCULINO,21/03/1961,04265770436,700206489800821,nao,sim,465,PARDA,MARIA DO CARMO DA SILVA,DAMIANA DE OLIVEIRA BEZERRA PEREIRA,ESF DOIS RIACHOS,https://afogadosdaingazeira.ephealth.com.br/po...
4,ALEXANDRA GONÇALVES PEREIRA,FEMININO,18/06/1979,--,700500706222652,sim,nao,--,PARDA,MARIA VERTUOSA DA SILVA GONÇALVES,MARIA DO SOCORRO GOES DE BRITO,ESF DOIS RIACHOS,https://afogadosdaingazeira.ephealth.com.br/po...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,VICENTE CARIRI DOS SANTOS,MASCULINO,15/04/1929,--,708007821543829,sim,nao,1301,PARDA,ESMERINDA ANTÔNIA DA CONCEIÇÃO,CECILIA MARIA DE OLIVEIRA,ESF DOIS RIACHOS,https://afogadosdaingazeira.ephealth.com.br/po...
419,VICENTE CIRILO DA SILVA,MASCULINO,20/09/1956,17130115491,700707989181671,nao,sim,1197,PARDA,MARGARIDA MARIA DE JESUS,CECILIA MARIA DE OLIVEIRA,ESF DOIS RIACHOS,https://afogadosdaingazeira.ephealth.com.br/po...
420,VILMA LUCIA BORGES SANTOS,FEMININO,01/07/1974,95310550178,709801061863795,sim,nao,743,BRANCA,MARIA DO CARMO BORGES SANTOS,GRACINALVA MARIA MIGUEL ATAIDE,ESF DOIS RIACHOS,https://afogadosdaingazeira.ephealth.com.br/po...
421,ZEZITO PEREIRA DA SILVA,MASCULINO,22/02/1967,--,705607453562112,sim,sim,114,PARDA,CORINA MARIA DA CONCEIÇAO,CRISTIANA FERREIRA DO NASCIMENTO,ESF DOIS RIACHOS,https://afogadosdaingazeira.ephealth.com.br/po...
